## N queen problem solved with Python

The N Queen is the problem of placing N chess queens on an N×N chessboard so that no two queens attack each other.

### Dependencies

In [31]:
import random
import numpy as np
from random import randint

### Variables

In [45]:
n = 8

mutationRate = 0.2  #out of 100
totalPopulation = 10  
generations = 1000

population = []

In [46]:
mutation_count = int(np.ceil(n* mutationRate))

In [47]:
#set of possible characters in the DNA
alpha_list = [x for x in range(0, n)] 
alpha_list

[0, 1, 2, 3, 4, 5, 6, 7]

### Define Functions

In [48]:
#get random character from the DNA 
def get_random_char():
    return random.choice(alpha_list)

#used for 0/1 toss
def random_toss():
    return random.choice([0,1])

In [49]:
#Create a random new chess set position
def new_set():
    return [get_random_char() for x in range(n)]

In [50]:
#shift the column in the chess set (used in comparision)
def roll(base,roll_by):
    if(roll_by > 0):
        return np.append(np.zeros(roll_by),base[0:n-roll_by])
    if(roll_by < 0):
        return np.append(base[-roll_by:],np.zeros(-roll_by))

In [56]:
#define fitness score for one chess set entity

#input  : [4,1,3,6,2,7,5,0]
#output : number between 0 and 1 -> closer to 1 means better position

#should measure perfect fitness for the solution
#for others, if should measure how far away from the solution it. 

def measure_fitness(child):
    
    #define variables
    score     = 0
    max_score = 0
    cube      = []
    
    #configure the cube (chess set representation)
    for i in range(n):
        base = np.zeros(n, dtype = int)
        base[child[i]] =1 
        base = base[::-1]
        cube.append(base)

    #sideways fitness
    #intution: Within each chess set positions, each columnar position can be occupied by only 1 queen. 
    #          So if duplicates are present, then queen is threated directly by another queen. 
    #Example : [1,2,3,4,5,6,7,7] => two queens at position 7 on 2 different columns
    #                               attacking each other
    #                               so score is 7/8. Includes 1 penalty for one threat.
    score += len(np.unique(child))
    max_score += n 


    #crossways fitness
    #intution: The rows above/below will be left/right shifted by distance between the rows. 
    #          And then compared with the initial row
    #Example : row 1 is [0,1,0]  
    #          row 2 is [1,0,0] 
    #          right/left shift row 2 by 1 position = [0,1,0] / [0,0,0]
    #          row 1 is exactly equal to right shift, means it is a threat for the queen.
    for i in range(n):
        for j in range(i+1,n):
            challenge = 0

            roll1 = roll(cube[j],j-i)   #right shift
            roll2 = roll(cube[j],i-j)   #left  shift
            
            if sum((cube[i] == roll2)*1) == 8:   #if right shift is exactly equal,
                challenge = 1                    #then queen is threated 
            if sum((cube[i] == roll1)*1) == 8:   #if left shift is exactly equal,
                challenge = 1                    #then queen is threated 
                 
            if challenge ==0:                    #if queen is not threatened
                score += 1                       #promote score
            max_score += 1                       

    #return the score as a percentage
    return round(score/max_score,3)  


#example
a = [1,2,3,4,5,6,7,0]
print("Fitness of ",a, "is:" , str(measure_fitness(a)) )

Fitness of  [1, 2, 3, 4, 5, 6, 7, 0] is: 0.389


In [57]:
#crossover function to create new

#input  : ([1,1,1,1,2,2,3,4],[5,5,4,4,6,6,1,1])
#output : [1,5,4,4,2,1,3,1]    

def breed(parent1,parent2):
    
    #Crossover
    child = []
    for i in range(n):
        
        toss = random_toss()
        if toss== 1:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    
    #Mutation
    for p in range(mutation_count):
        child[get_random_char()] = get_random_char()         

    return child

In [58]:
#fitness helper function
#returns fitness value
#used as a sorting key
def fn_fitness(x):
    return x[1]

In [59]:
#Survival
def survival():
    global population
    #evaluate fitness for all
    for i in range(len(population)):
        element = population[i][0]
        fitness = measure_fitness(element)
        population[i][1] = fitness
    

    #sort in order of fitness
    population.sort(key = fn_fitness, reverse = True)

    
    #Kill the weak 
    #Survival of the fittest
    population = population[:totalPopulation]
    

# MAIN CODE #######

In [60]:

#STEP 0: 
#create the initial population
population = []
for i in range(totalPopulation):
    population.append([new_set(),0])
    
# The Generation Loop
for i in range(1,generations):

    #STEP 1:
    #make babies
    for m in range(totalPopulation):
        for j in range(totalPopulation):

            parent1 = population[m][0]
            parent2 = population[j][0]
            child = breed(parent1,parent2)
            
            #add to general population
            population.append([child,0])   #initial fitness value = 0

    #STEP 2:
    #selection
    #"Survival of the fittest"
    #Survival using fitness score
    survival()
    
    
    
    #Print generation stats
    print("Generation No.: ", i)
    print("best fit :", population[0][1], population[0][0])
    print()
    
    #early stopping
    if population[0][1] == 1:
        break

Generation No.:  1
best fit : 0.917 [5, 0, 6, 3, 7, 1, 7, 0]

Generation No.:  2
best fit : 0.944 [7, 2, 0, 5, 6, 1, 3, 4]

Generation No.:  3
best fit : 0.944 [7, 2, 0, 5, 6, 1, 3, 4]

Generation No.:  4
best fit : 0.944 [7, 2, 0, 5, 6, 1, 3, 4]

Generation No.:  5
best fit : 0.944 [7, 2, 0, 5, 6, 1, 3, 4]

Generation No.:  6
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  7
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  8
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  9
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  10
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  11
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  12
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  13
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  14
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  15
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

Generation No.:  16
best fit : 0.972 [5, 2, 6, 3, 7, 1, 4, 0]

G